In [ ]:
import os
import re
import json
import joblib
import numpy as np
import pandas as pd

from pathlib import Path
from typing import Any
from dataclasses import dataclass, field
from datetime import datetime, timezone
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

pd.set_option("display.max_columns", 200)
np.random.seed(42)


In [ ]:
def find_project_root() -> Path:
    p = Path.cwd()

    for parent in [p] + list(p.parents):
        if (parent / "data").exists() and (parent / "notebooks").exists():
            return parent
        
    return p

def safe_read_csv(path: Path | None):
    if path is None:
        return None
    
    path = Path(path)
    if not path.exists():
        print("⚠️ not found:", path)
        return None
    
    return pd.read_csv(path, low_memory=False)

def latest_versioned_csv(folder: Path, base_name: str) -> Path | None:
    pattern = re.compile(rf"^{re.escape(base_name)}_v(\d+)\.csv$")
    best_v, best_path = None, None

    for f in folder.glob(f"{base_name}_v*.csv"):
        m = pattern.match(f.name)

        if m:
            v = int(m.group(1))

            if best_v is None or v > best_v:
                best_v, best_path = v, f

    return best_path

def next_versioned_file(folder: Path, base_name: str, ext: str = ".csv") -> Path:
    folder.mkdir(parents=True, exist_ok=True)

    pattern = re.compile(rf"^{re.escape(base_name)}_v(\d+){re.escape(ext)}$")
    versions = []

    for f in folder.glob(f"{base_name}_v*{ext}"):
        m = pattern.match(f.name)

        if m:
            versions.append(int(m.group(1)))

    v = (max(versions) + 1) if versions else 1

    return folder / f"{base_name}_v{v}{ext}"

def now_utc_iso() -> str:
    return datetime.now(timezone.utc).isoformat()

def run_stamp() -> str:
    return datetime.now().strftime("%Y%m%d_%H%M%S")

def _safe_str_path(p: Any) -> str | None:
    if p is None:
        return None
    try:
        return str(Path(p))
    except Exception:
        return str(p)


In [ ]:
def _parse_typed(v: str, default):
    """default의 타입을 기준으로 입력 문자열을 변환."""
    v = v.strip()
    if v == "":
        return default

    # None default면: 숫자로도, 문자열로도 받을 수 있게
    if default is None:
        # 숫자 시도
        try:
            if "." in v:
                return float(v)
            return int(v)
        except Exception:
            return v

    # bool
    if isinstance(default, bool):
        vv = v.lower()
        if vv in ("1", "true", "t", "y", "yes"):
            return True
        if vv in ("0", "false", "f", "n", "no"):
            return False
        return default

    # int/float
    if isinstance(default, (int, float, np.integer, np.floating)):
        try:
            if isinstance(default, float) or "." in v:
                return float(v)
            return int(v)
        except Exception:
            return default

    # 나머지는 문자열로
    return v


def prompt_dict_input(title: str, template: dict, defaults: dict | None = None) -> dict:
    """
    template: {key: None or default} 형태
    defaults: template에 덮어쓸 기본값(optional)
    """
    defaults = defaults or {}
    base = {**template, **defaults}  # defaults 우선

    print("\n" + "=" * 60)
    print(title)
    print("각 항목에 값을 입력하세요. (Enter: 기본값 사용)")
    print("=" * 60)

    out = {}
    for k, default in base.items():
        shown_default = "" if default is None else str(default)
        v = input(f"{k} [default={shown_default}]: ")
        out[k] = _parse_typed(v, default)
    return out


In [ ]:
@dataclass
class RunMetaLogger:
    project_root: Path
    run_type: str                          # "predict" / "train" / "shap"
    run_id: str = field(default_factory=run_stamp)
    meta_dir: Path = field(init=False)
    meta: dict[str, Any] = field(init=False)

    def __post_init__(self):
        self.meta_dir = self.project_root / "reports" / "metadata"
        self.meta_dir.mkdir(parents=True, exist_ok=True)

        self.meta = {
            "run_type": self.run_type,
            "run_id": self.run_id,
            "timestamp_local": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "timestamp_utc": now_utc_iso(),
            "inputs": {},
            "outputs": {},
            "notes": "",
        }

    def add_input(self, name: str, path: Any):
        p = None if path is None else Path(path)
        self.meta["inputs"][name] = {
            "path": _safe_str_path(p),
            "exists": (p.exists() if isinstance(p, Path) else False) if p is not None else None,
        }

    def add_inputs_from_globals(self, mapping: dict[str, str], g: dict[str, Any] | None = None):
        g = g if g is not None else globals()
        for meta_key, var_name in mapping.items():
            if var_name in g:
                self.add_input(meta_key, g[var_name])
            else:
                self.add_input(meta_key, None)

    def register_output(self, name: str, path: Any, extra: dict[str, Any] | None = None):
        p = None if path is None else Path(path)
        record: dict[str, Any] = {"path": _safe_str_path(p)}
        if p is not None:
            record["exists"] = p.exists()
            if p.exists() and p.is_file():
                record["size_bytes"] = p.stat().st_size
                record["modified_local"] = datetime.fromtimestamp(p.stat().st_mtime).strftime("%Y-%m-%d %H:%M:%S")
        if extra:
            record.update(extra)
        self.meta["outputs"][name] = record

    def save(self, latest_name: str | None = None) -> tuple[Path, Path | None]:
        snapshot_path = self.meta_dir / f"{self.run_type}_run_meta_{self.run_id}.json"
        with snapshot_path.open("w", encoding="utf-8") as f:
            json.dump(self.meta, f, ensure_ascii=False, indent=2)

        latest_path = None
        if latest_name is None:
            latest_name = f"{self.run_type}_run_meta_latest.json"
        if latest_name:
            latest_path = self.meta_dir / latest_name
            with latest_path.open("w", encoding="utf-8") as f:
                json.dump(self.meta, f, ensure_ascii=False, indent=2)

        return snapshot_path, latest_path


In [ ]:
PROJECT_ROOT = find_project_root()
print("CWD:", os.getcwd())
print("PROJECT_ROOT:", PROJECT_ROOT)

MODEL_DIR = PROJECT_ROOT / "models"
MODEL_DIR.mkdir(parents=True, exist_ok=True)
print("MODEL_DIR:", MODEL_DIR)

ACCUM_DIR = PROJECT_ROOT / "data" / "interim" / "01_daily_accumulated"
FEAT_DIR  = PROJECT_ROOT / "data" / "interim" / "02_comment_features"
RAW_DIR = PROJECT_ROOT / "data" / "raw" / "api"

PRED_DIR = PROJECT_ROOT / "reports" / "predictions"
PRED_DIR.mkdir(parents=True, exist_ok=True)

PATH_TRENDING_DAILY = ACCUM_DIR / "trending_videos_daily_kr.csv"
PATH_CHANNEL_DAILY  = ACCUM_DIR / "channels_daily_stats_kr.csv"
PATH_COMMENTS_RAW   = RAW_DIR / "comments_raw_kr.csv"

PATH_COMMENT_VIDEO_FEATS = FEAT_DIR / "comment_features_video_level_kr.csv"

PATH_CF_VIDEO_ID = PATH_COMMENT_VIDEO_FEATS
PATH_CF = None  # 두 번째 merge 스킵

print("PATH_TRENDING_DAILY:", PATH_TRENDING_DAILY)
print("PATH_CHANNEL_DAILY :", PATH_CHANNEL_DAILY)
print("PATH_COMMENTS_RAW  :", PATH_COMMENTS_RAW)
print("PATH_COMMENT_VIDEO_FEATS:", PATH_COMMENT_VIDEO_FEATS)

# 존재 체크
for p in [PATH_TRENDING_DAILY, PATH_CHANNEL_DAILY, PATH_COMMENTS_RAW]:
    if not p.exists():
        print("⚠️ not found:", p)


In [ ]:
# =========================
# Run Meta Logger 초기화
# - train: 모델 학습/저장 메타
# - predict: 예측 실행/산출물 메타
# =========================

train_logger = RunMetaLogger(project_root=PROJECT_ROOT, run_type="train")
train_logger.add_inputs_from_globals({
    "trending_daily": "PATH_TRENDING_DAILY",
    "channel_daily": "PATH_CHANNEL_DAILY",
    "comments_video_features": "PATH_COMMENT_VIDEO_FEATS",
    "comments_raw": "PATH_COMMENTS_RAW",
})

predict_logger = RunMetaLogger(project_root=PROJECT_ROOT, run_type="predict")
predict_logger.add_inputs_from_globals({
    "trending_daily": "PATH_TRENDING_DAILY",
    "channel_daily": "PATH_CHANNEL_DAILY",
    "comments_video_features": "PATH_COMMENT_VIDEO_FEATS",
    "comments_raw": "PATH_COMMENTS_RAW",
})

pred_path = PRED_DIR / f"pred_{predict_logger.run_id}.csv"


## 1) 영상 트렌딩 유지기간 예측 (video_id 단위 회귀)

In [ ]:
def build_trending_duration_dataset():
    trending = pd.read_csv(PATH_TRENDING_DAILY)

    # merge 안정성: video_id 타입 통일
    trending["video_id"] = trending["video_id"].astype(str)

    # 호환: 예전 누적 파일이 collected_date를 썼다면 date로 변환
    if "date" not in trending.columns and "collected_date" in trending.columns:
        trending = trending.rename(columns={"collected_date": "date"})

    # 날짜 tz 통일(UTC로 파싱 후 tz 제거 -> naive)
    trending["date"] = pd.to_datetime(trending["date"], errors="coerce", utc=True).dt.tz_localize(None)
    if "publish_date" in trending.columns:
        trending["publish_date"] = pd.to_datetime(trending["publish_date"], errors="coerce", utc=True).dt.tz_localize(None)

    # target: video_id별 트렌딩 유지기간(일)
    y = (trending.groupby("video_id")["date"]
                .nunique()
                .rename("trending_duration_days")
                .reset_index())

    # first day snapshot
    first_day = (trending.sort_values(["video_id", "date"])
                            .groupby("video_id", as_index=False)
                            .first())

    cols = ["video_id"]
    for c in ["channel_id", "region", "category_id", "publish_date", "date", "views", "likes", "comments"]:
        if c in first_day.columns:
            cols.append(c)

    first_day = first_day[cols].rename(columns={
        "views": "views_day1",
        "likes": "likes_day1",
        "comments": "comments_day1",
        "date": "first_trending_date"
    })

    # aggregates
    agg = (trending.groupby("video_id")
                    .agg(
                        views_mean=("views", "mean"),
                        views_max=("views", "max"),
                        likes_mean=("likes", "mean"),
                        likes_max=("likes", "max"),
                        comments_mean=("comments", "mean"),
                        comments_max=("comments", "max"),
                        trending_days=("date", "nunique"),
                    )
                    .reset_index())

    df = y.merge(first_day, on="video_id", how="left").merge(agg, on="video_id", how="left")

    # merge 이후 dtype 꼬임 방지: 다시 datetime 강제
    if "publish_date" in df.columns:
        df["publish_date"] = pd.to_datetime(df["publish_date"], errors="coerce", utc=True).dt.tz_localize(None)

    if "first_trending_date" in df.columns:
        df["first_trending_date"] = pd.to_datetime(df["first_trending_date"], errors="coerce", utc=True).dt.tz_localize(None)

    # days_to_first_trending
    df["days_to_first_trending"] = (
        (df["first_trending_date"] - df["publish_date"]).dt.total_seconds() / 86400.0
        if ("publish_date" in df.columns and "first_trending_date" in df.columns)
        else np.nan
    )

    # merge comment features (video-level)
    vcfv = safe_read_csv(PATH_CF_VIDEO_ID)
    if vcfv is not None and "video_id" in vcfv.columns:
        vcfv = vcfv.copy()
        vcfv["video_id"] = vcfv["video_id"].astype(str)
        vcfv = vcfv[~vcfv["video_id"].str.contains("#NAME", na=False)]
        vcfv = vcfv.drop_duplicates("video_id", keep="first")
        df = df.merge(vcfv.drop(columns=["category_name"], errors="ignore"), on="video_id", how="left")

    vcf = safe_read_csv(PATH_CF)
    if vcf is not None and "video_id" in vcf.columns:
        vcf = vcf.copy()
        vcf["video_id"] = vcf["video_id"].astype(str)
        vcf = vcf[~vcf["video_id"].str.contains("#NAME", na=False)]
        vcf = vcf.drop_duplicates("video_id", keep="first")
        df = df.merge(vcf, on="video_id", how="left", suffixes=("", "_vcf"))

    df.replace([np.inf, -np.inf], np.nan, inplace=True)

    # leakage 제거
    df.drop(columns=["trending_days"], errors="ignore", inplace=True)

    # raw/ID/원문 컬럼 제거
    DROP_COLS = ["comment_id", "comment_publishedAt", "text", "run_id", "category_name", "country", "likeCount"]
    df.drop(columns=[c for c in DROP_COLS if c in df.columns], inplace=True, errors="ignore")

    return df

df_video = build_trending_duration_dataset()
print("video dataset:", df_video.shape)
display(df_video.head(3))


In [ ]:
def train_trending_duration_model(df, test_size=0.2, random_state=42, prefix="trending_duration"):
    target_col = "trending_duration_days"

    if target_col not in df.columns:
        raise ValueError(f"'{target_col}' 컬럼이 df에 없습니다.")

    print("DEBUG MODEL_DIR =", MODEL_DIR)

    y = pd.to_numeric(df[target_col], errors="coerce")
    X = df.drop(columns=["video_id", target_col], errors="ignore").copy()
    X = X.replace([np.inf, -np.inf], np.nan)

    # 타깃 NaN 제거
    valid = y.notna()
    X = X.loc[valid].reset_index(drop=True)
    y = y.loc[valid].reset_index(drop=True)

    # datetime 컬럼 -> 숫자화(ts/dow)
    datetime_cols = list(X.select_dtypes(include=["datetime64[ns]", "datetime64[ns, UTC]"]).columns)
    for c in ["publish_date", "first_trending_date"]:
        if c in X.columns and c not in datetime_cols:
            X[c] = pd.to_datetime(X[c], errors="coerce", utc=True).dt.tz_localize(None)
            datetime_cols.append(c)

    datetime_cols = list(dict.fromkeys(datetime_cols))
    for c in datetime_cols:
        dt = pd.to_datetime(X[c], errors="coerce")
        ts = dt.astype("int64")
        ts = pd.Series(ts, index=X.index).where(dt.notna(), np.nan) / 1e9
        X[c + "_ts"] = ts
        X[c + "_dow"] = dt.dt.dayofweek

    if datetime_cols:
        X = X.drop(columns=datetime_cols, errors="ignore")

    # object지만 숫자열이면 numeric 변환
    for c in X.columns:
        if X[c].dtype == "object":
            tmp = pd.to_numeric(X[c], errors="coerce")
            if tmp.notna().mean() >= 0.9:
                X[c] = tmp

    # 전부 NaN 컬럼 제거
    all_nan_cols = X.columns[X.isna().all()].tolist()
    if all_nan_cols:
        print("Drop all-NaN cols:", all_nan_cols[:20], "..." if len(all_nan_cols) > 20 else "")
        X = X.drop(columns=all_nan_cols)

    # 상수 컬럼 제거
    nunique = X.nunique(dropna=True)
    const_cols = nunique[nunique <= 1].index.tolist()
    if const_cols:
        print("Drop constant cols:", const_cols[:20], "..." if len(const_cols) > 20 else "")
        X = X.drop(columns=const_cols)

    # cat/num 분리
    cat_cols = [c for c in X.columns if X[c].dtype == "object"]
    num_cols = [c for c in X.columns if c not in cat_cols]

    # 전처리 파이프라인
    num_pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])
    cat_pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ])

    pre = ColumnTransformer(
        transformers=[
            ("num", num_pipe, num_cols),
            ("cat", cat_pipe, cat_cols),
        ],
        remainder="drop"
    )

    model = RandomForestRegressor(
        n_estimators=600,
        random_state=random_state,
        n_jobs=-1,
        min_samples_leaf=2
    )

    pipe = Pipeline([("preprocess", pre), ("model", model)])

    # ==========================================
    # n_samples 작을 때 split 방어
    # ==========================================
    if len(y) < 10:
        print("⚠️ n_samples < 10 → holdout split 없이 전체 데이터로 학습(구조 검증 목적)")
        pipe.fit(X, y)

        # 평가 지표는 의미 없으니 None 처리
        mae = rmse = r2 = None

        print("[Channel Growth] fitted on all data (no test split)")
        print("Features used:", X.shape[1])
    else:
        X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

        pipe.fit(X_train, y_train)
        pred = pipe.predict(X_test)

        mae = mean_absolute_error(y_test, pred)
        rmse = mean_squared_error(y_test, pred) ** 0.5
        r2 = r2_score(y_test, pred)

        print(f"[Channel Growth] MAE: {mae:.4f}  RMSE: {rmse:.4f}  R2: {r2:.4f}")
        print("Features used:", X.shape[1])


    # 자동 저장
    model_path = next_versioned_file(MODEL_DIR, f"{prefix}_model", ext=".joblib")
    cols_path  = next_versioned_file(MODEL_DIR, f"{prefix}_feature_columns", ext=".joblib")

    print("DEBUG: about to save")
    print("DEBUG model_path =", model_path)
    print("DEBUG cols_path  =", cols_path)

    joblib.dump(pipe, model_path)
    joblib.dump(list(X.columns), cols_path)

    assert model_path.exists(), f"모델 저장 실패: {model_path}"
    assert cols_path.exists(),  f"컬럼 저장 실패: {cols_path}"

    metrics = {
    "mae": None if mae is None else float(mae),
    "rmse": None if rmse is None else float(rmse),
    "r2": None if r2 is None else float(r2),
    "n_samples": int(len(y)),
    "n_features": int(X.shape[1]),
    }

    print("saved:")
    print(" -", model_path)
    print(" -", cols_path)

    return pipe, list(X.columns), model_path, cols_path, metrics

video_model, video_feature_cols, video_model_path, video_cols_path, video_metrics = train_trending_duration_model(df_video)
print("RETURNED:", video_model_path, video_cols_path)


In [ ]:
def _latest_versioned_file(base_dir: str, base_name: str, ext: str = ".joblib") -> str:
    pattern = re.compile(rf"{re.escape(base_name)}_v(\d+){re.escape(ext)}$")
    best_v = None
    best_path = None

    for f in os.listdir(base_dir):
        m = pattern.match(f)

        if m:
            v = int(m.group(1))

            if best_v is None or v > best_v:
                best_v = v
                best_path = os.path.join(base_dir, f)

    if best_path is None:
        raise FileNotFoundError(f"{base_dir} 에 '{base_name}_v*.joblib' 파일이 없습니다.")
    
    return best_path

def _load_latest_model_and_cols(model_base_name: str, cols_base_name: str):
    model_path = _latest_versioned_file(MODEL_DIR, model_base_name)
    cols_path  = _latest_versioned_file(MODEL_DIR, cols_base_name)

    pipe = joblib.load(model_path)
    feature_cols = joblib.load(cols_path)

    return pipe, feature_cols, model_path, cols_path

def predict_trending_duration(input_dict: dict) -> float:
    pipe, feature_cols, model_path, cols_path = _load_latest_model_and_cols(
        "trending_duration_model",
        "trending_duration_feature_columns",
    )

    X_new = pd.DataFrame([input_dict]).copy()

    # 날짜 입력을 줬다면 학습 때와 동일하게 파생 생성
    for c in ["publish_date", "first_trending_date"]:
        if c in X_new.columns:
            dt = pd.to_datetime(X_new[c], errors="coerce", utc=True).dt.tz_localize(None)
            X_new[c + "_ts"] = dt.astype("int64") / 1e9
            X_new[c + "_dow"] = dt.dt.dayofweek

    # 학습 피처 컬럼에 맞추기 (없는 컬럼은 NaN)
    for c in feature_cols:
        if c not in X_new.columns:
            X_new[c] = np.nan

    X_new = X_new[feature_cols]
    pred = pipe.predict(X_new)[0]

    return float(pred)

def trending_duration_input_template():
    _, cols, _, _ = _load_latest_model_and_cols(
        "trending_duration_model",
        "trending_duration_feature_columns",
    )
    return {c: None for c in cols}

tpl = trending_duration_input_template()
print("template columns:", len(tpl))
print("loaded from:", _latest_versioned_file(MODEL_DIR, "trending_duration_model"))
list(tpl.keys())[:25]


In [ ]:
# =========================================
# Trending Duration 예측 (사용자 입력 or example)
# =========================================

example_video_input = {
    "category_id": 24,
    "views_day1": 120000,
    "likes_day1": 8000,
    "comments_day1": 1500,
    "views_mean": 180000,
    "views_max": 350000,
    "likes_mean": 12000,
    "likes_max": 22000,
    "comments_mean": 2000,
    "comments_max": 4200,
    "days_to_first_trending": 2.0,
}

use_example = input("Video Data (Enter: example 사용 / y: 직접 입력): ").strip().lower()
if use_example != "y":
    video_input = example_video_input
else:
    # 모델 템플릿(학습 피처) 기반으로 입력 받되, example 값으로 기본값 세팅
    tpl = trending_duration_input_template()
    # 여기서는 최소 항목만 받도록 subset 구성(원하면 tpl 전체로도 가능)
    subset_keys = list(example_video_input.keys())
    subset_tpl = {k: None for k in subset_keys}
    video_input = prompt_dict_input("Trending Duration 입력", subset_tpl, defaults=example_video_input)

pred_td = predict_trending_duration(video_input)
print("pred trending duration(days):", pred_td)


## 2) 채널 성장 예측 (channel_id×date 단위 회귀)
타깃: subscriber_growth_h = (h일 뒤 구독자수 - 오늘 구독자수). 기본 h=7

In [ ]:
def build_channel_growth_dataset(horizon_days=7):
    ch = pd.read_csv(PATH_CHANNEL_DAILY)

    # 날짜 파싱
    ch["date"] = pd.to_datetime(
        ch.get("date"), errors="coerce", utc=True
    ).dt.tz_localize(None)

    ch = ch.sort_values(["channel_id", "date"]).reset_index(drop=True)

    # 숫자형 강제
    for c in ["subscriber_count", "views_total", "video_count_total"]:
        if c in ch.columns:
            ch[c] = pd.to_numeric(ch[c], errors="coerce")

    # 채널별 관측 일수 사전 체크
    per_channel_days = ch.groupby("channel_id")["date"].nunique()

    need_days = horizon_days + 1
    eligible_channels = per_channel_days[per_channel_days >= need_days].index

    print(
        f"[channel coverage] channels={per_channel_days.shape[0]}, "
        f"rows={len(ch)}"
    )
    print(
        f"[channel coverage] horizon_days={horizon_days} → "
        f"need >= {need_days} days/channel"
    )
    print(
        f"[channel coverage] per-channel days "
        f"(min/median/max): "
        f"{per_channel_days.min()} / "
        f"{per_channel_days.median()} / "
        f"{per_channel_days.max()}"
    )
    print(
        f"[channel coverage] eligible channels: "
        f"{len(eligible_channels)} / {per_channel_days.shape[0]}"
    )

    if len(eligible_channels) == 0:
        raise ValueError(
            f"No channel has >= {need_days} days of data. "
            f"Current max days per channel = {per_channel_days.max()}. "
            f"Collect more daily data or lower horizon_days."
        )

    # 가능한 채널만 유지
    ch = ch[ch["channel_id"].isin(eligible_channels)].copy()

    # 타깃 생성
    ch["subscriber_future"] = (
        ch.groupby("channel_id")["subscriber_count"]
          .shift(-horizon_days)
    )
    ch["subscriber_growth_h"] = (
        ch["subscriber_future"] - ch["subscriber_count"]
    )

    # 변화량 피처
    ch["subs_delta_1d"] = (
        ch.groupby("channel_id")["subscriber_count"]
          .diff(1)
          .fillna(0)
    )

    if "views_total" in ch.columns:
        ch["views_delta_1d"] = (
            ch.groupby("channel_id")["views_total"]
              .diff(1)
              .fillna(0)
        )

    if "video_count_total" in ch.columns:
        ch["video_count_delta_1d"] = (
            ch.groupby("channel_id")["video_count_total"]
              .diff(1)
              .fillna(0)
        )

    # 7일 평균 추세
    ch["subs_delta_7d_mean"] = (
        ch.groupby("channel_id")["subs_delta_1d"]
          .rolling(7, min_periods=1)
          .mean()
          .reset_index(level=0, drop=True)
    )

    if "views_delta_1d" in ch.columns:
        ch["views_delta_7d_mean"] = (
            ch.groupby("channel_id")["views_delta_1d"]
              .rolling(7, min_periods=1)
              .mean()
              .reset_index(level=0, drop=True)
        )

    # 트렌딩(채널-일자) merge
    tr = pd.read_csv(PATH_TRENDING_DAILY)

    tr["date"] = pd.to_datetime(
        tr.get("date"), errors="coerce", utc=True
    ).dt.tz_localize(None)

    tr["channel_id"] = tr["channel_id"].astype(str)
    tr["video_id"] = tr["video_id"].astype(str)

    agg_dict = {"video_id": "nunique"}
    for c in ["views", "likes", "comments"]:
        if c in tr.columns:
            tr[c] = pd.to_numeric(tr[c], errors="coerce")
            agg_dict[c] = "sum"

    tr_agg = (
        tr.groupby(["channel_id", "date"], as_index=False)
          .agg(agg_dict)
          .rename(columns={
              "video_id": "trending_video_cnt",
              "views": "trending_views_sum",
              "likes": "trending_likes_sum",
              "comments": "trending_comments_sum",
          })
    )

    df = ch.merge(tr_agg, on=["channel_id", "date"], how="left")

    # 트렌딩 결측 = 0
    for c in df.columns:
        if c.startswith("trending_"):
            df[c] = df[c].fillna(0)

    # inf 처리
    df.replace([np.inf, -np.inf], np.nan, inplace=True)

    # 타깃 NaN 제거
    df = df[df["subscriber_growth_h"].notna()].reset_index(drop=True)

    print(f"[result] channel growth dataset shape: {df.shape}")

    return df

# =========================================
# Channel Growth dataset 생성 (입력 or 기본값)
# =========================================

h = input("Channel Growth Horizon Days (Enter = 1): ").strip()
HORIZON_DAYS = int(h) if h else 1

df_channel = build_channel_growth_dataset(horizon_days=HORIZON_DAYS)

print("channel dataset:", df_channel.shape)
display(df_channel.head(3))


In [ ]:
def train_channel_growth_model(df, test_size=0.2, random_state=42, prefix="channel_growth"):
    target_col = "subscriber_growth_h"

    if target_col not in df.columns:
        raise ValueError(f"'{target_col}' 컬럼이 df에 없습니다.")

    y = pd.to_numeric(df[target_col], errors="coerce")
    X = df.drop(columns=[target_col, "subscriber_future"], errors="ignore").copy()

    # ID 제거 (식별자 과적합 방지)
    X = X.drop(columns=["channel_id"], errors="ignore")

    # date -> dayofweek
    if "date" in X.columns:
        X["date"] = pd.to_datetime(X["date"], errors="coerce", utc=True).dt.tz_localize(None)
        X["dayofweek"] = X["date"].dt.dayofweek
        X = X.drop(columns=["date"], errors="ignore")

    # inf 제거
    X = X.replace([np.inf, -np.inf], np.nan)

    # 타깃 NaN 제거
    valid = y.notna()
    X = X.loc[valid].reset_index(drop=True)
    y = y.loc[valid].reset_index(drop=True)

    DROP_FOR_GROWTH = ["run_ts_utc", "channel_name", "created_date"]
    X = X.drop(columns=DROP_FOR_GROWTH, errors="ignore")
    
    # object인데 숫자열이면 numeric으로 바꾸기
    for c in X.columns:
        if X[c].dtype == "object":
            tmp = pd.to_numeric(X[c], errors="coerce")
            if tmp.notna().mean() >= 0.9:
                X[c] = tmp

    # 전부 NaN 컬럼 제거
    all_nan_cols = X.columns[X.isna().all()].tolist()
    if all_nan_cols:
        print("Drop all-NaN cols:", all_nan_cols[:20], "..." if len(all_nan_cols) > 20 else "")
        X = X.drop(columns=all_nan_cols)

    # 상수 컬럼 제거
    nunique = X.nunique(dropna=True)
    const_cols = nunique[nunique <= 1].index.tolist()
    if const_cols:
        X = X.drop(columns=const_cols)

    cat_cols = [c for c in X.columns if X[c].dtype == "object"]
    num_cols = [c for c in X.columns if c not in cat_cols]

    num_pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])
    cat_pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ])

    pre = ColumnTransformer([
        ("num", num_pipe, num_cols),
        ("cat", cat_pipe, cat_cols),
    ])
    model = RandomForestRegressor(
        n_estimators=900, random_state=random_state, n_jobs=-1, min_samples_leaf=2
    )

    pipe = Pipeline([("preprocess", pre), ("model", model)])

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    pipe.fit(X_train, y_train)
    pred = pipe.predict(X_test)

    mae = mean_absolute_error(y_test, pred)
    rmse = mean_squared_error(y_test, pred) ** 0.5
    r2 = r2_score(y_test, pred)

    print(f"[Channel Growth] MAE: {mae:.4f}  RMSE: {rmse:.4f}  R2: {r2:.4f}")
    print("Features used:", X.shape[1])

    # 자동 저장
    model_path = next_versioned_file(MODEL_DIR, f"{prefix}_model", ext=".joblib")
    cols_path  = next_versioned_file(MODEL_DIR, f"{prefix}_feature_columns", ext=".joblib")

    joblib.dump(pipe, model_path)
    joblib.dump(list(X.columns), cols_path)

    metrics = {
    "mae": None if mae is None else float(mae),
    "rmse": None if rmse is None else float(rmse),
    "r2": None if r2 is None else float(r2),
    "n_samples": int(len(y)),
    "n_features": int(X.shape[1]),
    }

    print("saved:")
    print(" -", model_path)
    print(" -", cols_path)

    return pipe, list(X.columns), model_path, cols_path, metrics


channel_model, channel_feature_cols, channel_model_path, channel_cols_path, channel_metrics = train_channel_growth_model(df_channel)
len(channel_feature_cols), channel_feature_cols[:25], channel_model_path, channel_cols_path


In [ ]:
# ==========================================
# 학습 메타 로그 저장
# ==========================================

# outputs: 모델 파일
train_logger.register_output("trending_duration_model", video_model_path, extra={"format": "joblib"})
train_logger.register_output("trending_duration_feature_columns", video_cols_path, extra={"format": "joblib"})
train_logger.meta.setdefault("metrics", {})["trending_duration"] = video_metrics

train_logger.register_output("channel_growth_model", channel_model_path, extra={"format": "joblib"})
train_logger.register_output("channel_growth_feature_columns", channel_cols_path, extra={"format": "joblib"})
train_logger.meta.setdefault("metrics", {})["channel_growth"] = channel_metrics
train_logger.meta.setdefault("params", {})["channel_growth_horizon_days"] = int(HORIZON_DAYS)

# snapshot/latest 파일명
train_meta_snapshot_path = train_logger.project_root / "reports" / "metadata" / f"train_run_meta_{train_logger.run_id}.json"
train_meta_latest_path   = train_logger.project_root / "reports" / "metadata" / "train_run_meta_latest.json"

train_snapshot_path, train_latest_path = train_logger.save()

train_logger.register_output("meta_snapshot", train_meta_snapshot_path, extra={"format": "json"})
train_logger.register_output("meta_latest", train_meta_latest_path, extra={"format": "json"})

train_logger.save()

print("✅ train meta snapshot:", train_snapshot_path)
print("✅ train meta latest  :", train_latest_path)


In [ ]:
def predict_channel_growth(input_dict: dict) -> float:
    pipe, feature_cols, model_path, cols_path = _load_latest_model_and_cols(
        "channel_growth_model",
        "channel_growth_feature_columns",
    )
    X_new = pd.DataFrame([input_dict]).copy()

    # date 지원(주면 dayofweek로 변환)
    if "date" in X_new.columns:
        dt = pd.to_datetime(X_new["date"], errors="coerce", utc=True).dt.tz_localize(None)
        X_new["dayofweek"] = dt.dt.dayofweek
        X_new = X_new.drop(columns=["date"], errors="ignore")

    for c in feature_cols:
        if c not in X_new.columns:
            X_new[c] = np.nan

    X_new = X_new[feature_cols]
    pred = pipe.predict(X_new)[0]

    return float(pred)

def channel_growth_input_template():
    _, cols, _, _ = _load_latest_model_and_cols(
        "channel_growth_model",
        "channel_growth_feature_columns",
    )
    return {c: None for c in cols}

tpl = channel_growth_input_template()
print("template columns:", len(tpl))
print("loaded from:", _latest_versioned_file(MODEL_DIR, "channel_growth_model"))
list(tpl.keys())[:25]


In [ ]:
# =========================================
# Channel Growth 예측 (사용자 입력 or example)
# =========================================

# 최신 모델의 "실제 입력 컬럼" 템플릿
tpl = channel_growth_input_template()

# 템플릿 기반 example (tpl 키만 사용)
example_channel_input = {
    "subscriber_count": 1_500_000,
    "views_total": 450_000_000,
    "video_count_total": 520,
    "views_delta_1d": 800_000,
    "views_delta_7d_mean": 700_000,
    "trending_views_sum": 2_000_000,
    "trending_likes_sum": 120_000,
    "trending_comments_sum": 9_000,
    "dayofweek": 3,
}

# tpl에 없는 키 제거 (안전장치)
example_channel_input = {k: example_channel_input.get(k, None) for k in tpl.keys()}

# -------------------------
# 입력 방식 선택
# -------------------------
use_example = input(
    "Channel Data (Enter: example 사용 / y: 직접 입력): "
).strip().lower()

if use_example != "y":
    ch_input = example_channel_input
else:
    ch_input = prompt_dict_input(
        title="Channel Growth 입력",
        template=tpl,
        defaults=example_channel_input
    )

# -------------------------
# 예측 실행
# -------------------------
pred_value = predict_channel_growth(ch_input)
print(f"pred subscriber growth in {HORIZON_DAYS} days:", pred_value)

# -------------------------
# 예측 결과 저장
# -------------------------
df_pred = pd.DataFrame([{
    **ch_input,
    f"pred_subscriber_growth_{HORIZON_DAYS}d": pred_value,
    "run_id": predict_logger.run_id,
    "timestamp_local": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
}])

df_pred.to_csv(pred_path, index=False, encoding="utf-8-sig")
print("✅ saved predictions:", pred_path)

# outputs 자동 기록
predict_logger.register_output(
    "predictions",
    pred_path,
    extra={
        "rows": int(len(df_pred)),
        "format": "csv"
    }
)

predict_logger.meta.setdefault("params", {})["channel_growth_horizon_days"] = int(HORIZON_DAYS)


In [ ]:
# ==========================================
# 예측 실행 메타 로그 저장
# ==========================================

meta_snapshot_path = predict_logger.project_root / "reports" / "metadata" / f"predict_run_meta_{predict_logger.run_id}.json"
meta_latest_path   = predict_logger.project_root / "reports" / "metadata" / "predict_run_meta_latest.json"

snapshot_path, latest_path = predict_logger.save()

predict_logger.register_output(
    "meta_snapshot",
    meta_snapshot_path,
    extra={"format": "json"}
)

predict_logger.register_output(
    "meta_latest",
    meta_latest_path,
    extra={"format": "json"}
)

# outputs까지 포함된 최종 메타 저장 (snapshot + latest)
predict_logger.save()

print("✅ meta snapshot:", snapshot_path)
print("✅ meta latest  :", latest_path)
